# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
import zipfile

orders_zip = zipfile.ZipFile('Orders.zip')
orders_zip.namelist() 
orders = pd.read_csv(orders_zip.open('Orders.csv'))

orders = orders.drop(columns='Unnamed: 0')

orders.head()

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# your code here
amout_client = orders.groupby(['Country','CustomerID'])['amount_spent'].sum()
amout_client = amout_client.reset_index()

amout_client['quartile'] = pd.qcut(amout_client['amount_spent'], 4, ['q1','q2','q3','q4'])
amout_client.head()

customer_label = {
        'q1': 'Minor',
        'q2': 'Regular',
        'q3': 'Preferred',
        'q4': 'VIP',
         }

amout_client['customer_type'] = amout_client['quartile']
amout_client = amout_client.replace({'customer_type':customer_label})

amout_client.head()

,Country,CustomerID,amount_spent,quartile,customer_type
0,Australia,12386,401.90,q2,Regular
1,Australia,12388,2780.66,q4,VIP
2,Australia,12393,1582.60,q3,Preferred
3,Australia,12415,124914.53,q4,VIP
4,Australia,12422,386.20,q2,Regular


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [9]:
# your code here
orders_country = amout_client.groupby(['Country','customer_type']).agg(quantity=('customer_type', 'count'))
orders_country = orders_country.reset_index()
orders_country[orders_country['customer_type']=='VIP'].sort_values(by='quantity', ascending=False).head()

,Country,customer_type,quantity
96,United Kingdom,VIP,934
41,Germany,VIP,39
37,France,VIP,29
12,Belgium,VIP,11
88,Switzerland,VIP,9


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [22]:
vip_preferred = amout_client[(amout_client["customer_type"] == "VIP") | (amout_client["customer_type"] == "Preferred")].groupby("Country").agg({"CustomerID": "count"})
vip_preferred.sort_values("CustomerID", ascending= False).head()

,CustomerID
Country,
United Kingdom,1910
Germany,61
France,50
Belgium,18
Spain,18
